In [1]:
from drn_interactions.interactions.loaders import SpontaneousActivityLoader
from drn_interactions.io import load_derived_generic
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.seqnmf import SeqNMF
import pandas as pd
from drn_interactions.config import Config

c:\Users\roryl\.conda\envs\drn38\lib\site-packages\outdated\utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
neuron_types = load_derived_generic('neuron_types.csv')
sessions = neuron_types["session_name"].dropna().unique()

res = []
Ks = [6]
Ls = [10,]
bin_widths = [0.05, 0.1, 0.5, 1, 2]

loader_fac = lambda session_name, bin_width, shuffle: SpontaneousActivityLoader(session_name=session_name, bin_width=bin_width, shuffle=shuffle)
preprocessor_fac = lambda: InteractionsPreprocessor(z=False, minmax=True, gaussian_sigma=None)
model_fac = lambda K, L, Lambda: SeqNMF(K, L, Lambda)

for shuffle in (True, False):
    for session in sessions:
        for k in Ks:
            for l in Ls:
                for bin_width in bin_widths:
                    loader = loader_fac(session, bin_width, shuffle)
                    preprocessor = preprocessor_fac()
                    model = model_fac(k, l, 0.1)
                    spikes = preprocessor(loader())
                    score = model.sequenciness_score(spikes.T)
                    res.append({
                        "session_name": session,
                        "k": k,
                        "L": l,
                        "bin_width": bin_width,
                        "score": score,
                        "shuffle": shuffle,
                    })


ESHOCK_03_LOC1
ESHOCK_04_LOC1
ESHOCK_06_LOC1
ESHOCK_07_LOC1
ESHOCK_08_LOC1
ESHOCK_09_LOC1
hamilton_10
hamilton_03
hamilton_04
hamilton_09
hamilton_31
hamilton_38
hamilton_37
hamilton_35
hamilton_36
hamilton_32
acute_15
acute_16
acute_01
acute_14
acute_12
acute_11
ESHOCK_03_LOC1
ESHOCK_04_LOC1
ESHOCK_06_LOC1
ESHOCK_07_LOC1
ESHOCK_08_LOC1
ESHOCK_09_LOC1
hamilton_10
hamilton_03
hamilton_04
hamilton_09
hamilton_31
hamilton_38
hamilton_37
hamilton_35
hamilton_36
hamilton_32
acute_15
acute_16
acute_01
acute_14
acute_12
acute_11


In [4]:
df_out = pd.DataFrame.from_records(res)
dd = Config.derived_data_dir

df_out.to_csv(dd / "seqnmf_sequenciness_scores.csv")